<a href="https://colab.research.google.com/github/ukomiljon/Coursera_Capstone/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Part 1**

In [ ]:
import pandas as pd
import numpy as np
import requests

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

df = pd.read_html(url)[0]
df=df[df.Neighborhood != 'Not assigned']
df.reset_index(drop=True,inplace=True)  
df.head() 

In [ ]:
df=df.groupby(['Postal Code']).first() 
df.reset_index(inplace=True) 
df.columns=["Postal Code", "Borough","Neighborhood"]  
df

In [ ]:
df[df['Borough'] == 'Not assigned']

There were not exist any duplicate values on "Postal Code" and no 'Not assigned' in column 'Borough'

In [ ]:
df.shape

**Part 2**

Installing geocoder

In [ ]:
!pip install geocoder

Reading locations from the given url

In [ ]:
import geocoder
geo_url = "http://cocl.us/Geospatial_data"

geoloc_df = pd.read_csv(geo_url)
geoloc_df.head()

In [ ]:
Inner join two dataframes

In [ ]:
result = pd.merge(df,geoloc_df,on='Postal Code')
result

Final result

In [ ]:
result.head()